In [1]:
from utils import DATA
import config
xdf_path = r'C:\\EEGAnotator\Data\\Patients\\F10\\PictureNaming\\xdf\\f10.xdf'
edf_path = r'C:\\EEGAnotator\Data\\Patients\\F10\\PictureNaming\\eeg\\f10.edf'

In [2]:
data = DATA(filepath_xdf=xdf_path, filepath_edf=edf_path)
data.print_info()

*********************************************************************************
***************************Loading .edf file***************************
Extracting EDF parameters from C:\EEGAnotator\Data\Patients\F10\PictureNaming\eeg\f10.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*******************************Completed*******************************
Checking for Interruptions
No interruptions detected.
*********************************************************************************
***************************Loading .xdf file***************************
*******************************Completed*******************************
***************************EEG File Info***************************
Filepath: C:\\EEGAnotator\Data\\Patients\\F10\\PictureNaming\\eeg\\f10.edf
EEG ::: Start Time: 2022-06-28 10:06:08+00:00, End Time:2022-06-28 11:45:36.500000+00:00
Number of Channels: 148
Sampling Frequency: 512.0
Number of data points: 3055872
No. o

In [5]:
data.audio_streams.marker_times

array(['2022-06-28T09:13:39', '2022-06-28T09:13:39',
       '2022-06-28T09:13:40', ..., '2022-06-28T09:41:31',
       '2022-06-28T09:41:33', '2022-06-28T09:41:33'],
      dtype='datetime64[s]')

In [4]:
data.audio_streams.marker_time_gaps_items

array(['2022-06-28T09:18:29', '2022-06-28T09:19:17',
       '2022-06-28T09:29:01', '2022-06-28T09:31:24',
       '2022-06-28T09:37:41'], dtype='datetime64[s]')

In [6]:
data.audio_streams.markers

[['ExperimentStarted'],
 ['StartBlockSaying'],
 ['StartReading:TENEDOR'],
 ['EndReading:TENEDOR'],
 ['StartSaying:TENEDOR'],
 ['EndSaying:TENEDOR'],
 ['StartReading:OREJA'],
 ['EndReading:OREJA'],
 ['StartSaying:OREJA'],
 ['EndSaying:OREJA'],
 ['StartReading:CUCHILLO'],
 ['EndReading:CUCHILLO'],
 ['StartSaying:CUCHILLO'],
 ['EndSaying:CUCHILLO'],
 ['StartReading:TOMATE'],
 ['EndReading:TOMATE'],
 ['StartSaying:TOMATE'],
 ['EndSaying:TOMATE'],
 ['StartReading:LEÓN'],
 ['EndReading:LEÓN'],
 ['StartSaying:LEÓN'],
 ['EndSaying:LEÓN'],
 ['StartReading:LECHUGA'],
 ['EndReading:LECHUGA'],
 ['StartSaying:LECHUGA'],
 ['EndSaying:LECHUGA'],
 ['StartReading:CUCHARA'],
 ['EndReading:CUCHARA'],
 ['StartSaying:CUCHARA'],
 ['EndSaying:CUCHARA'],
 ['StartReading:MANO'],
 ['EndReading:MANO'],
 ['StartSaying:MANO'],
 ['EndSaying:MANO'],
 ['StartReading:COPA'],
 ['EndReading:COPA'],
 ['StartSaying:COPA'],
 ['EndSaying:COPA'],
 ['StartReading:NARANJA'],
 ['EndReading:NARANJA'],
 ['StartSaying:NARANJA'],
 

In [34]:
def find_inconsistencies(data):
    inconsistencies = []
    for i in range(len(data)-3):
        if data[i][0].startswith('StartReading'):
            if not (data[i+1][0].startswith('EndReading') and 
                    data[i+2][0].startswith('StartSaying') and 
                    data[i+3][0].startswith('EndSaying')):
                inconsistencies.append(data[i:i+4])
    return inconsistencies

# Example usage:
print(find_inconsistencies(data.audio_streams.markers))  # Output: []

def check_order(data):
    start_index = None
    end_index = None

    # Find the index of the first StartReading
    for i, item in enumerate(data):
        if item[0].startswith('StartReading:'):
            start_index = i
            break

    # Find the index of the last EndSaying
    for i, item in enumerate(data[::-1]):
        if item[0].startswith('EndSaying:'):
            end_index = len(data) - i - 1
            break

    # If either start or end index is not found, return False
    if start_index is None or end_index is None:
        return False
    
    word_order = {}
    data = data[start_index:end_index ]
    print(start_index, end_index, len(data))
    for item in data:
        print(item)
        event = item[0].split(':')
        word = event[1]
        action = event[0]
        if word not in word_order:
            word_order[word] = []
        word_order[word].append(action)

    for word, actions in word_order.items():
        if actions != ['StartReading', 'EndReading', 'StartSaying', 'EndSaying']:
            return False
    return word_order

[]


In [33]:
check_order(data.audio_streams.markers)

2 1030 1028
['StartReading:TENEDOR']
['EndReading:TENEDOR']
['StartSaying:TENEDOR']
['EndSaying:TENEDOR']
['StartReading:OREJA']
['EndReading:OREJA']
['StartSaying:OREJA']
['EndSaying:OREJA']
['StartReading:CUCHILLO']
['EndReading:CUCHILLO']
['StartSaying:CUCHILLO']
['EndSaying:CUCHILLO']
['StartReading:TOMATE']
['EndReading:TOMATE']
['StartSaying:TOMATE']
['EndSaying:TOMATE']
['StartReading:LEÓN']
['EndReading:LEÓN']
['StartSaying:LEÓN']
['EndSaying:LEÓN']
['StartReading:LECHUGA']
['EndReading:LECHUGA']
['StartSaying:LECHUGA']
['EndSaying:LECHUGA']
['StartReading:CUCHARA']
['EndReading:CUCHARA']
['StartSaying:CUCHARA']
['EndSaying:CUCHARA']
['StartReading:MANO']
['EndReading:MANO']
['StartSaying:MANO']
['EndSaying:MANO']
['StartReading:COPA']
['EndReading:COPA']
['StartSaying:COPA']
['EndSaying:COPA']
['StartReading:NARANJA']
['EndReading:NARANJA']
['StartSaying:NARANJA']
['EndSaying:NARANJA']
['ExperimentResting']


IndexError: list index out of range

In [21]:
data.audio_streams.markers[:4]

[['ExperimentStarted'],
 ['StartBlockSaying'],
 ['StartReading:TENEDOR'],
 ['EndReading:TENEDOR']]

In [12]:
import numpy as np

def calculate_time_gaps(time_array, time_interval):
    differences = np.diff(time_array).astype('int')
    indices = np.where(differences > time_interval)[0]

    time_gaps = differences[indices]
    corresponding_items = time_array[1:][indices]
    
    difference_groups = []
    start_idx = 0
    for idx in indices:
        group = time_array[start_idx:idx+1]
        if differences[idx] > time_interval:
            difference_groups.append(group)
        start_idx = idx + 1
    
    return time_gaps, corresponding_items, difference_groups


In [13]:
tg, ci, dg = calculate_time_gaps(data.audio_streams.marker_times, 3)

In [14]:
tg

array([ 10,   4, 539,   5,   5])

In [15]:
ci

array(['2022-06-28T09:18:29', '2022-06-28T09:19:17',
       '2022-06-28T09:29:01', '2022-06-28T09:31:24',
       '2022-06-28T09:37:41'], dtype='datetime64[s]')

In [18]:
dg[0].shape

(258,)